In [213]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [214]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL

In [215]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.rename(columns={'id_professor(a)': 'id_professor'}, inplace=True)
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [216]:
''' 
Assumindo que o número total de aulas por materia no semestre é 40 
''' 

total_de_aulas = 40

In [217]:
### Calculando taxa de frequencia por materia e semestre

In [221]:
# importa tabela de estudantes
tabela_estudantes = pd.read_excel('files/tabela_estudante.xlsx', index_col=False) # Importando tabela de alunos
tabela_estudantes.set_index('Unnamed: 0', inplace=True)
tabela_estudantes.index.rename('id', inplace=True)

In [219]:
total_faltas_alunos = []
taxa_de_falta_por_semestre_e_materias = []

for i in range(1, tabela_estudantes.shape[0] + 1):
    query = f'select * from notas where id_estudante = {i}'
    notas_aluno = pd.read_sql(query, engine)
    for j in range(1):
        taxa_de_falta_por_semestre_e_materias.append((notas_aluno['faltas']/total_de_aulas))

In [220]:
taxa_de_falta_por_semestre_e_materias = pd.DataFrame(taxa_de_falta_por_semestre_e_materias).fillna(0)
taxa_de_falta_por_semestre_e_materias.index = tabela_estudantes['nome']
taxa_de_falta_por_semestre_e_materias = taxa_de_falta_por_semestre_e_materias.replace(0, '-')
taxa_de_falta_por_semestre_e_materias.index.rename('Estudantes', inplace=True)
taxa_de_falta_por_semestre_e_materias.to_excel('files/files_questions/question_8/taxa_de_falta_por_materia_de_cada_aluno.xlsx')